# Full IV Masurement 

This code runs a suit of measurements completely automatically, it communicates with a `PR59` PID temperature controller through serial port and with a Digilent `AD2` device.
The measurement is done bringing the PID-controlled thermostat at a given temperature and then running a series of IV curve measurements, all measurements can be done both forward and backwards to see if theres any kind of hysteresis or discrepancy.

## Hardware Parameters
* `port` ⇒ it is the connection port to the PID controller, be sure it is set correctly
* `R` ⇒ The Shunt resistance through which the current is measured
* `sigR` ⇒ Error in the Resistance value, necessary to perform uncertanties calculations
* `expected_Vj` ⇒ Expected junction voltage of whatever diode is being tested, only used for plotting reasons

## Measurement Parameters
### Temperature
* `max_stabilizing_time` ⇒ Maximum time to be waited for any given temperature to stabilize, used to avoid getting stuck on any specific measurement
* `nT`  ⇒ Number of temperature points to be measured
* `T_range` ⇒ Range of temperatures in whihc to carry on the measurements ( <b> <u> IMPORTANT: The code will error out if the nT and T_range are such that the temperatures to measure have decimals different than .0 or .5 because the PID controller has problems with too many decimal places </u> </b> )
* `flag_T_return` ⇒ if set True every measurements will be repeated twice, once going from the low extreme of the range to the high and one going backwords, otherwise only rising
* `admitted_deviation` ⇒ The admitted deviation of the actual temperature from the desired temperatured, in percentage of the desired temperature( e.g. 0.05 means an acceptable range of 5% over or under the desired temp)
* `stabilized_samples` ⇒ The number of Temperature measurements that have to fall completely withing the acceptable window in order to start the IV measurements
### Voltage
* `n_pulls` ⇒  The number of measurement to be done for each temperature
* `measuring_delay` ⇒  The delay( in seconds) between successive IV measurements at the same temperature
* `nv` ⇒  Number of voltage points to measure the current at
* `V_range` ⇒ Range of the applied voltage 
* `flag_return_V` ⇒ Same as `flag_T_return` but regarding applied voltage and not applied temperature

## Plot Parameters
* `flag_plot` ⇒ If set to True the Data anf information are plotted while the measurements are taking place( <b> WARNING: it SIGNIFICANTLY increase runtime </b>)

## Save Parameters
* `flag_save` ⇒ It decides if the Data is saved or not, there are multiple error and checkpoints to advise when set to false

## Testing Parameters
* `flag_testing` ⇒ If set to True the Serial connection is never initialized and fake temperature values are used







In [6]:
import serial
import time
import numpy as np
import tdwf
import matplotlib.pyplot as plt
import scipy.optimize as so
import psutil, os, sys
import warnings

# -[Pre-Processing work]-------------------------------------------
p = psutil.Process(os.getpid())
os_used = sys.platform
if os_used == "win32":  # Windows (either 32-bit or 64-bit)
    p.nice(psutil.REALTIME_PRIORITY_CLASS)
elif os_used == "linux":  # linux
    p.nice(psutil.IOPRIO_HIGH)
else:  # MAC OS X or other
    p.nice(-20)  



# -[Parameters]-------------------------------------------

# 1. hardware parameters
port = 'COM7'  # serial port for the PR59
R = 9.971e3 # shunt resistance [Ohm]
sigR = 299 # shunt resistance tolerance [Ohm]
expected_Vj = 0.7 # expected junction Voltage [V]

# 2. Measurement parameters
max_stabilizing_time = 1800 # max stabilizing time in seconds 300s = 5min, 1800s = 30min
nT = 61 # number of temperature points to measure
T_range = [10, 70] # temperature range [C]
temperatures = np.linspace(T_range[0], T_range[1], nT) # temperatures at which to measure
for i in temperatures: # we are not sure how many decimal places the device can handle, dont be silly and use only 0.5 steps
    if i % 0.5 != 0: raise ValueError(f"ONLY INTEGER VALUES ARE ALLOWED UNTIL FURTHER TESTING") 
flag_return_T = True # if true the measurement are done both increasing and lowering the temperature
admitted_deviation = 0.05 # [% of desired temp] maximum deviation from the set temperature to be considered stable
stabilized_samples = 20 # number of temperature samples to be stabilized before measuring the IV curve

n_pulls = 3 # number of pulls to be done for each temperature
measuring_delay = 10 # [s] delay between the pulls
nv = 50 # number of voltage points to measure
V_range = [-1, 5] # voltage range [V]
offsets = np.linspace(V_range[0], V_range[1], nv) # voltages at which to measure
flag_return_V = True # if true the IV charatestics is measured in both directions

# 3. Plot Parameters
flag_show = True # Plotting the data while measuring

# 4. Data Saving Parameters
flag_save = True # Saving the data in a file, not sure why you should ever turn it off but ehy, you can
if not flag_save: warnings.warn("Data will not be saved, are you REALLY sure?")

# 5. Testing parameters
flag_testing = True # if true the serial connection will not run and fake temperature data will be generated
if flag_testing: warnings.warn("Testing mode is on, ya really sure?") # raising a warning to remind the user that the data are not real



# -[Serial Connection initialization and startup]-------------------------------------------
if not flag_testing:
    # 1. Connection setup
    ser = serial.Serial()
    ser.port = port
    ser.baudrate = 115200   
    ser.timeout = 1
    ser.rts = False
    ser.dtr = False
    ser.open()

    # 2. Communcation function definition
    # sending and receiving data to the device
    def query(message, flagR=True, flagW=True):
        if flagW:    
            message += '\n\r'
            ser.write(message.encode('utf-8'))
            res = ser.readline().decode()
        if flagR:
            res = ser.readline().decode()
            return res
        
    # 3. Testing the connection
    swap = query("$V?")
    print('\n'+str(swap)+'\n')
    time.sleep(1)


# -[PID Configuration]-------------------------------------------
# technically should be already done in device, leave here just in case


# -[AD2 Connection initialization and startup]-------------------------------------------
# 1. Connection
ad2 = tdwf.AD2()
print(f"\n")

# 2. Function generator setup
wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.func = tdwf.funcDC
wavegen.w1.start()

# 3. Analog input setup
scope = tdwf.Scope(ad2.hdwf)
scope.fs = 1e6
scope.npt = 8192
scope.ch1.rng = 10
scope.ch2.rng = 10


# -[Saving pipeline setup]-------------------------------------------
if flag_save and not flag_testing:
    Directory = f"IV-T_dependence_{time.strftime('%Y%m%d_%H%M%S')}" # directory name based on the current date and time
    parent_dir = "../../../Data"
    path = os.path.join(parent_dir, Directory) # creating the directory
    os.mkdir(path)
    print(f"Data will be saved in {path}\n")
    log_file = os.path.join(path, f"log.txt")   #creating the log file


# -[Setting Recap]-------------------------------------------
print(f'Voltages to be measured = {offsets}')
print(f'Number of pulls per Voltage = {n_pulls}\n')
print(f'Temperatures where to measure = {temperatures}\n')

print(f'Status flags:')
print(f'  - save = {flag_save}')
print(f'  - show = {flag_show}')
print(f'  - testing = {flag_testing}')
print(f'  - return_T = {flag_return_T}')
print(f'  - return_V = {flag_return_V}')




C:\Users\monca\AppData\Local\Temp\ipykernel_17788\2484802485.py:57: UserWarning: Testing mode is on, ya really sure?
  if flag_testing: warnings.warn("Testing mode is on, ya really sure?") # raising a warning to remind the user that the data are not real


Dispositivo #1 [SN:210321B1F3F8, hdwf=1] connesso!
Configurazione #1


Voltages to be measured = [-1.         -0.87755102 -0.75510204 -0.63265306 -0.51020408 -0.3877551
 -0.26530612 -0.14285714 -0.02040816  0.10204082  0.2244898   0.34693878
  0.46938776  0.59183673  0.71428571  0.83673469  0.95918367  1.08163265
  1.20408163  1.32653061  1.44897959  1.57142857  1.69387755  1.81632653
  1.93877551  2.06122449  2.18367347  2.30612245  2.42857143  2.55102041
  2.67346939  2.79591837  2.91836735  3.04081633  3.16326531  3.28571429
  3.40816327  3.53061224  3.65306122  3.7755102   3.89795918  4.02040816
  4.14285714  4.26530612  4.3877551   4.51020408  4.63265306  4.75510204
  4.87755102  5.        ]
Number of pulls per Voltage = 3

Temperatures where to measure = [10. 11. 12. 13. 14. 15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27.
 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40. 41. 42. 43. 44. 45.
 46. 47. 48. 49. 50. 51. 52. 53. 54. 55. 56. 57. 58. 59. 60. 61. 62. 63.
 64. 65. 6

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


# -[Plot initialization]-------------------------------------------
if flag_show:
    fig, [[ax1, ax3, ax5], [ax2, ax4a, ax6]] = plt.subplots(2, 3, figsize=(10, 8), dpi=100, tight_layout=True, width_ratios=[1, 3, 1], height_ratios=[1, 1])
    ax4b = ax4a.twinx()
    fig.canvas.manager.set_window_title('IV Measurement - T dependence')

# -[Operator communication function]-------------------------------------------
def communicate(message, flag_plot=True, flag_type=True, flag_log=True):
    if flag_show and flag_plot:
        fig.suptitle(message, fontsize=18)
    if flag_type:
        print(message)
    if flag_log and not flag_testing and flag_save:
        with open(path+"/log.txt", "a") as log:
            log.write(time.strftime('%H%M%S') + ' >>> ' + message + "\n")
            log.close()

# -[Temperature Measurement Function]-------------------------------------------
def measure_temp():
    if flag_testing:
        data = np.sin(len(temp)/10) * .10 + np.sin(len(temp)/2) * .05 + temp_desired# test data with obvious pattern
    else:
        data = '\r\n'
        while(data == '\r\n' or data == ''):
            data = query("$R100?")
        data = float(data)
    
    temp.append(data)
    return data


# -[IV Measurement Function]-------------------------------------------
def measureIV(list_offsets):
    offsets = list_offsets.copy()
    flag_first = True
    Vd = np.full((nv, 2), np.nan) # diode voltage data
    ErrVd = np.full((nv, 2), np.nan) # diode voltage error
    Vr = np.full((nv, 2), np.nan) # resistor voltage data
    ErrVr = np.full((nv, 2), np.nan) # resistor voltage error
    Id = np.full((nv, 2), np.nan) # current data
    ErrId = np.full((nv, 2), np.nan) # current error
    Td = np.full((nv, 2), np.nan) # temperature data

    for ar_V in range(2 if flag_return_V else 1):
        for ii in range(nv):
            if ar_V == 0:
                i = ii
            else:
                i = nv - ii - 1

            # 1. Set voltage
            wavegen.w1.offs = offsets[i]
            wavegen.w1.start()

            # 2. Measure temperature
            measure_temp()
            update_Tplots()

            # 2. Voltage sampling and automatic fitting
            scope.sample()

            fitfuncDC = lambda x, o: np.array( [o for i in x] )
            pp1, cov1 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch1.vals, p0=[wavegen.w1.offs])
            pp2, cov2 = so.curve_fit(fitfuncDC, scope.time.vals, scope.ch2.vals, p0=[wavegen.w1.offs])

            Td[i, ar_V] = temp[-1] # temperature
            Vd[i, ar_V] = pp1[0] # diode voltage
            ErrVd[i, ar_V] = np.sqrt(cov1[0][0]) # diode voltage error
            Vr[i, ar_V] = pp2[0] # resistor voltage
            ErrVr[i, ar_V] = np.sqrt(cov2[0][0]) # resistor voltage error
            Id[i, ar_V] = pp2[0] / R # current
            ErrId[i, ar_V] = np.sqrt( cov2[0][0]/R**2 + sigR**2 * pp2[0]**2 / R**4 ) # current error

            if flag_show:
                if flag_first:
                    flag_first = False

                    hp1_true, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                    hp1_sym, = ax1.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp1), "--", label="Fit Ch1", color="#00e7e7")
                    ax1.grid(linestyle='-.')
                    ax1.set_ylabel("Vd [V]", fontsize=15)
                    ax1.set_ylim([-1.2*max(offsets), 1.2*max(offsets)])
                    ax1.set_title(f"Time plots", fontsize=15)


                    hp2_true, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                    hp2_sym, = ax2.plot(1000*scope.time.vals, fitfuncDC(1000*scope.time.vals, *pp2), "--", label="Fit Ch2", color="#e500a7")
                    ax2.grid(linestyle='-.')
                    ax2.set_xlabel("Time [msec]", fontsize=15)
                    ax2.set_ylabel("Vr [A]", fontsize=15)
                    ax2.set_ylim([-1.2*max(offsets), 1.2*max(offsets)])


                    hp3A, = ax3.plot(Vd[:, 0], Id[:, 0], ".", markerfacecolor = "none", label="Go Run", color="tab:purple")
                    if flag_return_V:
                        hp3R, = ax3.plot(Vd[:, 1], Id[:, 1], "v",  markerfacecolor = "none", label="Return Run", color="tab:cyan")
                    
                    ax3.grid(linestyle='-.')
                    ax3.set_xlabel("Vd [V]", fontsize=15)
                    ax3.set_ylabel("Id = Vr/R [mA]", fontsize=15)
                    ax3.set_xlim([min(offsets)-0.5, expected_Vj])
                    ax3.set_ylim([(min(offsets)/R-0.2/R)*1e3, max(offsets)/R*1e3])
                    ax3.legend(loc='upper left')
                    ax3.set_title(f"I-V Charateristic", fontsize=15)


                    hp4VA, = ax4a.plot(offsets, Vd[:, 0], ".", markerfacecolor = "none", label="Vd Go", color="tab:orange")
                    if flag_return_V:
                        hp4VR, = ax4a.plot(offsets, Vd[:, 1], "v",  markerfacecolor = "none", label="Vd Return", color="tab:orange")                
                    ax4a.grid(linestyle='-.')
                    ax4a.set_xlabel("Vcc [V]", fontsize=15)
                    ax4a.yaxis.label.set_color("tab:orange")
                    ax4a.set_ylabel("Vd [V]", fontsize=15)
                    ax4a.legend(loc='upper left')
                    ax4a.tick_params(axis='y', labelcolor="tab:orange")
                    ax4a.set_xlim([min(offsets)-0.5, 1.2*max(offsets)])
                    ax4a.set_ylim([min(offsets)-0.5, 1.2*expected_Vj])

                    
                    hp4IA, = ax4b.plot(offsets, Id[:, 0]*1e3, ".", markerfacecolor = "none", label="Id Go", color="tab:blue")
                    if flag_return_V:
                        hp4IR, = ax4b.plot(offsets, Id[:, 1]*1e3, "v",  markerfacecolor = "none", label="Id Return", color="tab:blue")
                    ax4b.yaxis.label.set_color("tab:blue")
                    ax4b.yaxis.set_label_position("right")
                    ax4b.set_ylabel("Id = Vr/R [mA]", fontsize=15)
                    ax4b.legend(loc='lower right')
                    ax4b.tick_params(axis='y', labelcolor="tab:blue")
                    ax4b.set_ylim([(min(offsets)/R-0.2/R)*1e3, max(offsets)/R*1e3])

                else:
                    hp1_true.set_xdata(1000*scope.time.vals)
                    hp1_true.set_ydata(scope.ch1.vals)
                    hp1_sym.set_xdata(1000*scope.time.vals)
                    hp1_sym.set_ydata(fitfuncDC(scope.time.vals, *pp1))
                    hp2_true.set_xdata(1000*scope.time.vals)
                    hp2_true.set_ydata(scope.ch2.vals)
                    hp2_sym.set_xdata(1000*scope.time.vals)
                    hp2_sym.set_ydata(fitfuncDC(scope.time.vals, *pp2))
                    if ar_V==0:
                        hp3A.set_xdata(Vd[:, 0])
                        hp3A.set_ydata(Id[:, 0]*1e3)

                        hp4VA.set_xdata(offsets[:])
                        hp4VA.set_ydata(Vd[:, 0])
                        hp4IA.set_xdata(offsets[:])
                        hp4IA.set_ydata(Id[:, 0]*1e3)
                    else:
                        hp3R.set_xdata(Vd[:, 1])
                        hp3R.set_ydata(Id[:, 1]*1e3)

                        hp4VR.set_xdata(offsets[:])
                        hp4VR.set_ydata(Vd[:, 1])
                        hp4IR.set_xdata(offsets[:])
                        hp4IR.set_ydata(Id[:, 1]*1e3)

    # data saving
    if flag_save and not flag_testing:
        info = f"IV Measurement - T = {temp_desired:.2f} C - Pull number {pull}\n"
        if ar_V == 0:
            data_to_save = np.c_[Td[:, 0], offsets, Vd[:, 0], ErrVd[:, 0], Vr[:, 0], ErrVr[:, 0], Id[:, 0], ErrId[:, 0]]
            info = info + f"Td_go[C]\tVcc\tVd_go[V]\tErrVd_go[V]\tVr_go[V]\tErrVr_go[V]\tId_go[A]\tErrId_go[A]\n"
        else:
            data_to_save = np.c_[Td, offsets, Vd, ErrVd, Vr, ErrVr, Id, ErrId]
            info = info + f"Td_go[C]\tTd_return[C]\tVcc\tVd_go[V]\tVd_return[V]\tErrVd_go[V]\tErrVd_return[V]\tVr_go[V]\tVr_return[V]\tErrVr_go[V]\tErrVr_return[V]\tId_go[A]\tId_return[A]\tErrId_go[A]\tErrId_return[A]\n"

        info = info + ' - T Return Leg'*(ar_T==1) + ' - T Go Leg'*(ar_T==0)
        filename = f"IV_T{temp_desired:.2f}_V{offsets[0]:.2f}_{offsets[-1]:.2f}_{pull}Pull"
        if ar_T == 0:
            filename += "_TGo"
        else:
            filename += "_TReturn"

        np.savetxt(path+'/'+filename+".txt", data_to_save, delimiter="\t", header=info, fmt="%s")

    wavegen.w1.stop()
    return Td, Vd, ErrVd, Vr, ErrVr, Id, ErrId


# -[Temperature Plots Update]---------------------------------------------
def update_Tplots():
    if flag_show:
        ax5.clear()
        ax6.clear()

        ax5.set_title("Full data acquisition")
        ax5.plot(np.arange(0, len(temp)), temp, "-", label="Temperature", color="tab:blue")
        ax5.plot(np.linspace(0, len(temp), len(temp)), np.ones(len(temp))*temp_desired, "--", label="Setpoint", color="k")
        ax5.plot(np.linspace(0, len(temp), len(temp)), np.ones(len(temp))*(temp_desired + admitted_deviation*temp_desired), "--", label="Window", color="tab:red")
        ax5.legend(loc='upper right')
        ax5.set_xlabel("Sample", fontsize=15)
        ax5.yaxis.set_label_position("right")
        ax5.yaxis.tick_right()
        ax5.set_ylabel("Temperature [C]", fontsize=15)
        ax5.grid(linestyle='-.')
        

        secondGraphMax = min(stabilized_samples, len(temp))

        ax6.set_title(f"Latest {stabilized_samples} samples\nLatest temp {temp[-1]:.2f} C")
        ax6.plot(np.arange(len(temp)-secondGraphMax, len(temp)), temp[-secondGraphMax:], "-", label="Latest Temp Data", color="tab:orange")
        ax6.plot(np.arange(len(temp)-secondGraphMax, len(temp)), np.ones(secondGraphMax)*temp_desired, "--", label="Desired Temp", color="k")
        ax6.set_xlabel("Sample", fontsize=15)
        ax6.yaxis.set_label_position("right")
        ax6.yaxis.tick_right()
        ax6.set_ylabel("Temperature [C]", fontsize=15)
        ax6.legend(loc='upper right')
        ax6.grid(linestyle='-.')

        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.show(block=False)




temp = []
# -[Main Loop]-------------------------------------------
if flag_testing:
    communicate("\n\n\n\bTESTING TESTING TESTING TESTING TESTING TESTING TESTING\b\n\n\n", flag_plot=False, flag_type=False, flag_log=True)
communicate(input("Please insert a \bMEANINGFUL\b comment to understand what the measurement in about: ") + '\n', flag_plot=False, flag_type=False, flag_log=True)
for ar_T in range(2 if flag_return_T else 1):
    if ar_T == 1:
        temperatures = np.flip(temperatures)


    for temp_desired in temperatures:
        temp_skip = False
        # 1. Set temperature
        temp = []
        if not flag_testing:
            query(f"$R0={(temp_desired)}")
        
        communicate(f'------------------- {temp_desired:.2f} C - ' + 'Go'*(ar_T==0) + 'Ret'*(ar_T==1) +' ----------------------', flag_plot=False)
        communicate(f'Changing temperature to {temp_desired:.2f} C - Waiting for stabilization' + ' - Return Leg'*(ar_T==1) + ' - Go Leg'*(ar_T==0))

        starting_waiting_time = time.time()
        while True:
            # 2. Wait for temperature to stabilize
            measure_temp()


            if(len(temp) >= stabilized_samples):
                if abs(max(temp[-stabilized_samples:]) - temp_desired) < admitted_deviation*temp_desired and abs(min(temp[-stabilized_samples:]) - temp_desired) < admitted_deviation*temp_desired:
                    break
            

            update_Tplots()

            if time.time() - starting_waiting_time > max_stabilizing_time:
                # avoid measures and go to next temperature
                temp_skip = True
                communicate(f"Temperature {temp_desired:.2f} C not reached - skipping")
                break

        if(temp_skip):
            continue
        
        pull = 0
        starting_measuring_time = time.time()
        while pull < n_pulls:
            if time.time() - starting_measuring_time > measuring_delay or pull == 0:
                if flag_show:
                    ax1.clear()
                    ax2.clear()
                    ax3.clear()
                    ax4a.clear()
                    ax4b.clear()
                communicate(f'IV Measurement - target T = {temp_desired:.2f} C - actual T = {temp[-1]:.2f} C - Pull number {pull}' + ' - Return Leg'*(ar_T==1) + ' - Go Leg'*(ar_T==0))
                measureIV(offsets)
                pull += 1
                starting_measuring_time = time.time()
                if pull != n_pulls: communicate(f'Waiting for next measurement at T = {temp_desired:.2f} C' + ' - Return Leg'*(ar_T==1) + ' - Go Leg'*(ar_T==0))
            else:
                measure_temp()
                update_Tplots()

communicate(f"\n\nALL MEASUREMENTS DONE DUDE\n\n")

ad2.close()
print("Connection closed")    









------------------- 10.00 C - Go ----------------------
Changing temperature to 10.00 C - Waiting for stabilization - Go Leg
IV Measurement - target T = 10.00 C - actual T = 10.09 C - Pull number 0 - Go Leg
Waiting for next measurement at T = 10.00 C - Go Leg
IV Measurement - target T = 10.00 C - actual T = 10.07 C - Pull number 1 - Go Leg
Waiting for next measurement at T = 10.00 C - Go Leg
IV Measurement - target T = 10.00 C - actual T = 9.97 C - Pull number 2 - Go Leg


KeyboardInterrupt: 

In [26]:
ser.close()

NameError: name 'ser' is not defined

In [ ]:
ad2.close()

Dispositivo disconnesso.
